In [1]:
import pandas as pd
import dtale as dt
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Dataset DS01 - All Lending Club loan

Lending Club es una compañía estadounidense de préstamo entre particulares, con sede en San Francisco. Fue el primer sitio de préstamo entre particulares en registrar sus productos como valores con la Securities and Exchange Commission, y ofrecer un servicio de préstamos en el mercado secundario

Read data

In [2]:
ds01_accepted = pd.read_csv("../../data/raw/DS01_LC_accepted_2007_to_2018Q4.csv", sep=",")
ds01_rejected = pd.read_csv("../../data/raw/DS01_LC_rejected_2007_to_2018Q4.csv", sep=",")

In [3]:
ds01_accepted.shape

(2260701, 151)

In [4]:
ds01_rejected.shape

(27648741, 9)

In [ ]:
# Test dvc

display(ds01_accepted.head(5))
ds01_accepted.drop(columns=['id', 'member_id'], inplace=True)
display(ds01_accepted.head(5))

# Rewrite dataset
ds01_accepted.to_csv("../../data/raw/DS01_LC_accepted_2007_to_2018Q4.csv",index=False)

# Re-read
ds01_accepted_test = pd.read_csv("../../data/raw/DS01_LC_accepted_2007_to_2018Q4.csv", sep=",")
ds01_accepted_test.head(1)

# Verify changes in dvc by cmd command "dvc status"
# To revert the change - git checkout HEAD ~1 PATH_FILE.dvc and then dvc checkout

# Read again file to check the changes are reverted
ds01_accepted_test = pd.read_csv("../../data/raw/DS01_LC_accepted_2007_to_2018Q4.csv", sep=",")
ds01_accepted_test.head(1)

# Success!!

Metadata understanding

In [6]:
# Using dtale library to get information of dataframe

# Dataset: ds01_accepted
accepted = dt.show(ds01_accepted)
accepted.open_browser()

# Dataset: ds01_rejected
rejected = dt.show(ds01_rejected)
rejected.open_browser()

2024-07-15 16:40:55,317 - ERROR    - Exception occurred while processing request: '<' not supported between instances of 'str' and 'int'
Traceback (most recent call last):
  File "c:\Users\elisa\OneDrive\MÁSTER\TFM\.venv\Lib\site-packages\dtale\views.py", line 120, in _handle_exceptions
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\elisa\OneDrive\MÁSTER\TFM\.venv\Lib\site-packages\dtale\views.py", line 2312, in describe
    uniq_vals[uniq_vals["count"] == top_freq].sort_values("value").head(5)
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\elisa\OneDrive\MÁSTER\TFM\.venv\Lib\site-packages\pandas\core\frame.py", line 7200, in sort_values
    indexer = nargsort(
              ^^^^^^^^^
  File "c:\Users\elisa\OneDrive\MÁSTER\TFM\.venv\Lib\site-packages\pandas\core\sorting.py", line 439, in nargsort
    indexer = non_nan_idx[non_nans.argsort(kind=kind)]
                          ^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError:

# Dataset DS02 - Default of Credit Card Clients

Data from a research that aimed at the case of customers' default payments in Taiwan and compares the predictive accuracy of probability of default among six data mining methods.

In [13]:
# pip install ucimlrepo

from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
default_of_credit_card_clients = fetch_ucirepo(id=350) 
  
# data (as pandas dataframes) 
X = default_of_credit_card_clients.data.features 
y = default_of_credit_card_clients.data.targets 
  
# metadata 
print(default_of_credit_card_clients.metadata) 
  
# variable information 
print(default_of_credit_card_clients.variables) 


{'uci_id': 350, 'name': 'Default of Credit Card Clients', 'repository_url': 'https://archive.ics.uci.edu/dataset/350/default+of+credit+card+clients', 'data_url': 'https://archive.ics.uci.edu/static/public/350/data.csv', 'abstract': "This research aimed at the case of customers' default payments in Taiwan and compares the predictive accuracy of probability of default among six data mining methods.", 'area': 'Business', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 30000, 'num_features': 23, 'feature_types': ['Integer', 'Real'], 'demographics': ['Sex', 'Education Level', 'Marital Status', 'Age'], 'target_col': ['Y'], 'index_col': ['ID'], 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2009, 'last_updated': 'Fri Mar 29 2024', 'dataset_doi': '10.24432/C55S3H', 'creators': ['I-Cheng Yeh'], 'intro_paper': {'title': 'The comparisons of data mining techniques for the predictive accuracy of probability of default of cre

In [19]:
X

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23
0,20000,2,2,1,24,2,2,-1,-1,-2,...,689,0,0,0,0,689,0,0,0,0
1,120000,2,2,2,26,-1,2,0,0,0,...,2682,3272,3455,3261,0,1000,1000,1000,0,2000
2,90000,2,2,2,34,0,0,0,0,0,...,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000
3,50000,2,2,1,37,0,0,0,0,0,...,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000
4,50000,1,2,1,57,-1,0,-1,0,0,...,35835,20940,19146,19131,2000,36681,10000,9000,689,679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,220000,1,3,1,39,0,0,0,0,0,...,208365,88004,31237,15980,8500,20000,5003,3047,5000,1000
29996,150000,1,3,2,43,-1,-1,-1,-1,0,...,3502,8979,5190,0,1837,3526,8998,129,0,0
29997,30000,1,2,2,37,4,3,2,-1,0,...,2758,20878,20582,19357,0,0,22000,4200,2000,3100
29998,80000,1,3,1,41,1,-1,0,0,0,...,76304,52774,11855,48944,85900,3409,1178,1926,52964,1804


In [24]:
ds02 = pd.read_excel("../../data/raw/DS02_default_of_credit_card_clients.xls", engine = 'xlrd')
ds02

,Unnamed: 0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X15,X16,X17,X18,X19,X20,X21,X22,X23,Y
0,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
1,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
2,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29996,29996,220000,1,3,1,39,0,0,0,0,...,88004,31237,15980,8500,20000,5003,3047,5000,1000,0
29997,29997,150000,1,3,2,43,-1,-1,-1,-1,...,8979,5190,0,1837,3526,8998,129,0,0,0
29998,29998,30000,1,2,2,37,4,3,2,-1,...,20878,20582,19357,0,0,22000,4200,2000,3100,1
29999,29999,80000,1,3,1,41,1,-1,0,0,...,52774,11855,48944,85900,3409,1178,1926,52964,1804,1


# Dataset DS03 - Home Credit Default Risk

Home Credit is an international consumer finance provider with operations in multiple European and Asian countries. They focus on responsible lending primarily to people with little or no credit history.

Read data

In [7]:
ds03_application_test = pd.read_csv("../../data/raw/DS03_application_test.csv", sep=",")
ds03_application_train= pd.read_csv("../../data/raw/DS03_application_train.csv", sep=",")

In [41]:
ds03_bureau_balance = pd.read_csv("../../data/raw/DS03_bureau_balance.csv", sep=",")
ds03_bureau= pd.read_csv("../../data/raw/DS03_bureau.csv", sep=",")

ds03_credit_card_balance = pd.read_csv("../../data/raw/DS03_credit_card_balance.csv", sep=",")
ds03_installments_payments= pd.read_csv("../../data/raw/DS03_installments_payments.csv", sep=",")

ds03_pos_cash = pd.read_csv("../../data/raw/DS03_POS_CASH_balance.csv", sep=",")
ds03_previous_applications= pd.read_csv("../../data/raw/DS03_previous_application.csv", sep=",")

ds03_sample_submission= pd.read_csv("../../data/raw/DS03_sample_submission.csv", sep=",")

In [45]:
ds03_bureau_balance

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C
...,...,...,...
27299920,5041336,-47,X
27299921,5041336,-48,X
27299922,5041336,-49,X
27299923,5041336,-50,X


In [44]:
ds03_sample_submission

,SK_ID_CURR,TARGET
0,100001,0.5
1,100005,0.5
2,100013,0.5
3,100028,0.5
4,100038,0.5
...,...,...
48739,456221,0.5
48740,456222,0.5
48741,456223,0.5
48742,456224,0.5


# German credit risk

In [25]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
statlog_german_credit_data = fetch_ucirepo(id=144) 
  
# data (as pandas dataframes) 
X = statlog_german_credit_data.data.features 
y = statlog_german_credit_data.data.targets 
  
# metadata 
print(statlog_german_credit_data.metadata) 
  
# variable information 
print(statlog_german_credit_data.variables) 


{'uci_id': 144, 'name': 'Statlog (German Credit Data)', 'repository_url': 'https://archive.ics.uci.edu/dataset/144/statlog+german+credit+data', 'data_url': 'https://archive.ics.uci.edu/static/public/144/data.csv', 'abstract': 'This dataset classifies people described by a set of attributes as good or bad credit risks. Comes in two formats (one all numeric). Also comes with a cost matrix', 'area': 'Social Science', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 1000, 'num_features': 20, 'feature_types': ['Categorical', 'Integer'], 'demographics': ['Other', 'Marital Status', 'Age', 'Occupation'], 'target_col': ['class'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1994, 'last_updated': 'Thu Aug 10 2023', 'dataset_doi': '10.24432/C5NC77', 'creators': ['Hans Hofmann'], 'intro_paper': None, 'additional_info': {'summary': 'Two datasets are provided.  the original dataset, in the form provided by

In [26]:
X

,Attribute1,Attribute2,Attribute3,Attribute4,Attribute5,Attribute6,Attribute7,Attribute8,Attribute9,Attribute10,Attribute11,Attribute12,Attribute13,Attribute14,Attribute15,Attribute16,Attribute17,Attribute18,Attribute19,Attribute20
0,A11,6,A34,A43,1169,A65,A75,4,A93,A101,4,A121,67,A143,A152,2,A173,1,A192,A201
1,A12,48,A32,A43,5951,A61,A73,2,A92,A101,2,A121,22,A143,A152,1,A173,1,A191,A201
2,A14,12,A34,A46,2096,A61,A74,2,A93,A101,3,A121,49,A143,A152,1,A172,2,A191,A201
3,A11,42,A32,A42,7882,A61,A74,2,A93,A103,4,A122,45,A143,A153,1,A173,2,A191,A201
4,A11,24,A33,A40,4870,A61,A73,3,A93,A101,4,A124,53,A143,A153,2,A173,2,A191,A201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,A14,12,A32,A42,1736,A61,A74,3,A92,A101,4,A121,31,A143,A152,1,A172,1,A191,A201
996,A11,30,A32,A41,3857,A61,A73,4,A91,A101,4,A122,40,A143,A152,1,A174,1,A192,A201
997,A14,12,A32,A43,804,A61,A75,4,A93,A101,4,A123,38,A143,A152,1,A173,1,A191,A201
998,A11,45,A32,A43,1845,A61,A73,4,A93,A101,4,A124,23,A143,A153,1,A173,1,A192,A201
